# Continous Growth - Chordia Model with exclusion of outliers

Same model as in version 3. However this time I have corrected for outliers that exceed the 0.9999 and the 0.0001 quantile levels.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.stats.stattools import durbin_watson

In [2]:
pd.set_option('use_inf_as_na', True)

In [3]:
df=pd.read_excel("BBREIT-1.07.xlsx",sheet_name="PROPORTIONAL_BID_ASK_SPREAD", index_col="Dates")
df.dropna(axis=1,inplace=True)

In [4]:
for i in df:
    df[i]= np.log(df[i])-np.log(df[i]).shift(1)

C:\Users\ravn_\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\ravn_\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
df.drop(index=df.index[0], axis=0, inplace=True)

In [6]:
df.isnull().values.sum()

55

In [7]:
df.dropna(inplace=True)

In [8]:
df.isnull().values.sum()

0

In [9]:
q =df.quantile(0.9999)
b =df.quantile(0.0001)

In [10]:
df=df[(df < q) & (df > b)]

In [11]:
df["Sum"]=df.sum(axis=1)

In [12]:
Coefficients = []
Pvalues = []
Rsquared_adj = []
Bpagan = []
DBWatson = []
STD = []

for i in df:
    df["VOL"]= ((df[i]-df[i].shift(1))/df[i].shift(1))**2 # Squared return variable as we spoke about
    df["M"]=((df["Sum"]-df[i])/157) # Cross Sectional Mean Variable
    df["Mlag"]=df["M"].shift(1)
    df["Mlead"]=df["M"].shift(-1)
    df.replace([np.inf], np.nan, inplace=True)
    df=df.fillna(0)
        
        
      
    y = df[i]
    x = df[["M","VOL","Mlag","Mlead"]]
    x = sm.add_constant(x)
    model = sm.OLS(y,x, missing = 'drop') 
    results = model.fit()
    print(results.summary())
    print(sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x))
    print(durbin_watson(results.resid))
    
    
    # Filling the lists with the content specified
    coefficients=results.params 
    Coefficients.append(coefficients)
    
    pvalues=results.pvalues
    Pvalues.append(pvalues)
    
    rsquared_adj = results.rsquared_adj
    Rsquared_adj.append(rsquared_adj)
    
    bpagan = sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x)
    Bpagan.append(bpagan)
    
    std=results.bse
    STD.append(std)
    
    dbwatson = durbin_watson(results.resid)
    DBWatson.append(dbwatson)
    
    # CLearing the dynamic variables after use. Mostly for practical reuse-reasons
    Drop=df[["M","VOL","Mlag","Mlead"]]
    df.drop(columns=Drop, inplace=True)

                            OLS Regression Results                            
Dep. Variable:          AMT US Equity   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.039
Method:                 Least Squares   F-statistic:                     6.213
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           6.91e-05
Time:                        12:03:32   Log-Likelihood:                -952.17
No. Observations:                 516   AIC:                             1914.
Df Residuals:                     511   BIC:                             1936.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0167      0.068     -0.245      0.8

(6.598123906172436, 0.15871183102126937, 1.654705976894102, 0.15925807884835316)
2.753781203773102
                            OLS Regression Results                            
Dep. Variable:          DRE US Equity   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     10.45
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           3.84e-08
Time:                        12:03:33   Log-Likelihood:                -440.31
No. Observations:                 516   AIC:                             890.6
Df Residuals:                     511   BIC:                             911.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

(2.081315781850254, 0.7208057182588453, 0.5173738556244075, 0.7230047171727331)
2.9131939277012995
                            OLS Regression Results                            
Dep. Variable:          KIM US Equity   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     17.91
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           8.96e-14
Time:                        12:03:33   Log-Likelihood:                 192.57
No. Observations:                 516   AIC:                            -375.1
Df Residuals:                     511   BIC:                            -353.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

(12.993121478826529, 0.011309452572269365, 3.299897754480144, 0.010998629196254495)
2.94106699248759
                            OLS Regression Results                            
Dep. Variable:           FR US Equity   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     5.337
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           0.000323
Time:                        12:03:33   Log-Likelihood:                -610.40
No. Observations:                 516   AIC:                             1231.
Df Residuals:                     511   BIC:                             1252.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

(4.892574981511068, 0.2984986320973298, 1.2228866639247726, 0.30006809553121055)
2.691686791514865
                            OLS Regression Results                            
Dep. Variable:          CUZ US Equity   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     5.498
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           0.000244
Time:                        12:03:33   Log-Likelihood:                -483.95
No. Observations:                 516   AIC:                             977.9
Df Residuals:                     511   BIC:                             999.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

...

(3.6363763209513786, 0.4574471233605748, 0.9066745833083097, 0.459727405892996)
2.777304729834614
                            OLS Regression Results                            
Dep. Variable:          GNL US Equity   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     12.10
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           2.09e-09
Time:                        12:03:34   Log-Likelihood:                -340.61
No. Observations:                 516   AIC:                             691.2
Df Residuals:                     511   BIC:                             712.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

(1.890086990323936, 0.755965789515253, 0.46966340641119086, 0.758024783780986)
2.964734016534279
                            OLS Regression Results                            
Dep. Variable:          RTL US Equity   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                     7.007
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           1.70e-05
Time:                        12:03:34   Log-Likelihood:                -213.52
No. Observations:                 516   AIC:                             437.0
Df Residuals:                     511   BIC:                             458.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

(16.966349066206956, 0.001962265891202146, 4.343296467386922, 0.0018337753753288038)
2.9351668444999253
                            OLS Regression Results                            
Dep. Variable:         CLDT US Equity   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     5.492
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           0.000246
Time:                        12:03:34   Log-Likelihood:                -493.86
No. Observations:                 516   AIC:                             997.7
Df Residuals:                     511   BIC:                             1019.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

(1.2040014604388465, 0.877439364990102, 0.2987808510699591, 0.8787466670046196)
2.8761541650643325
                            OLS Regression Results                            
Dep. Variable:         CMCT US Equity   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.792
Date:                Sat, 22 Oct 2022   Prob (F-statistic):              0.129
Time:                        12:03:34   Log-Likelihood:                -682.65
No. Observations:                 516   AIC:                             1375.
Df Residuals:                     511   BIC:                             1397.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

C:\Users\ravn_\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


## Summary Results

Average coefficient much more like previous models when correcting for outliers. R2 reduced to about 6 per cent. Average P-value raised to about 9 per cent, Breuch Pagan significance level dropped by about 8 per cent. Durbin Watson test statistic remained very much the same (expectatly so). 

In [13]:
Coef=pd.DataFrame(Coefficients)
round(Coef["M"].mean(),4)

0.8048

In [14]:
R2=pd.DataFrame(Rsquared_adj)
round(R2.mean(),4)

0    0.0565
dtype: float64

In [15]:
DBW=pd.DataFrame(DBWatson)
round(DBW.mean(),4)

0    2.8631
dtype: float64

In [16]:
PV=pd.DataFrame(Pvalues)
round(PV["M"].mean(),4)

0.0894

In [17]:
BP=pd.DataFrame(Bpagan)
round(BP[1].mean(),4)

0.3421

In [18]:
PV.describe()

,const,M,VOL,Mlag,Mlead
count,159.000000,1.570000e+02,1.590000e+02,158.000000,158.000000
mean,0.774288,8.937964e-02,9.711067e-02,0.442003,0.437946
std,0.178291,1.997502e-01,2.094815e-01,0.295439,0.297274
min,0.210979,1.108731e-15,1.231362e-23,0.000453,0.000006
25%,0.684584,3.507795e-05,6.890853e-05,0.194044,0.184890
50%,0.811722,4.608027e-03,6.784196e-03,0.380115,0.427766
75%,0.923128,4.534829e-02,5.901480e-02,0.689986,0.672088
max,0.999140,9.955671e-01,9.790165e-01,0.998155,0.969062


In [21]:
round(PV["M"].median(),4)

0.0046

In [22]:
round(PV["M"].max(),4)

0.9956